In [54]:
import pandas as pd
import numpy as np
from pathlib2 import Path
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def write_to_submission_file(predicted_labels, out_file, target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels, index = np.arange(1, predicted_labels.shape[0] + 1), columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [3]:
PATH_TO_DATA = Path('../../data/alice/')

times = ['time%s' % i for i in range(1, 11)]
train_df = pd.read_csv(PATH_TO_DATA / 'train_sessions.csv',
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv(PATH_TO_DATA / 'test_sessions.csv',
                      index_col='session_id', parse_dates=times)

In [97]:
train_df['time1'].min(), train_df['time1'].max(), test_df['time1'].min(), test_df['time1'].max()

(Timestamp('2013-01-12 08:05:57'),
 Timestamp('2014-04-30 23:39:53'),
 Timestamp('2014-05-01 17:14:03'),
 Timestamp('2014-12-05 23:26:53'))

In [4]:
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [3]:
# !!!!!!!!
try_train_df = try_train_df.sort_values(by='time1')

try_train_df = train_df[train_df['time1'] >= '2014-01-01 00:00:00']

In [4]:
train_df = train_df.sort_values(by='time1')

In [10]:
#train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null datetime64[ns]
site2     81308 non-null float64
time2     81308 non-null datetime64[ns]
site3     80075 non-null float64
time3     80075 non-null datetime64[ns]
site4     79182 non-null float64
time4     79182 non-null datetime64[ns]
site5     78341 non-null float64
time5     78341 non-null datetime64[ns]
site6     77566 non-null float64
time6     77566 non-null datetime64[ns]
site7     76840 non-null float64
time7     76840 non-null datetime64[ns]
site8     76151 non-null float64
time8     76151 non-null datetime64[ns]
site9     75484 non-null float64
time9     75484 non-null datetime64[ns]
site10    74806 non-null float64
time10    74806 non-null datetime64[ns]
dtypes: datetime64[ns](10), float64(9), int64(1)
memory usage: 13.3 MB


In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

In [6]:
class DataPreparator(BaseEstimator, TransformerMixin):
    """
    Fill NaN with zero values.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        sites = ['site%s' % i for i in range(1, 11)]
        return X[sites].fillna(0).astype('int')

In [7]:
class ListPreparator(BaseEstimator, TransformerMixin):
    """
    Prepare a CountVectorizer friendly 2D-list from data.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.values.tolist()
        # Convert dataframe rows to strings
        return [" ".join([str(site) for site in row]) for row in X]

In [34]:
class AttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new attributes to training and test set.
    """
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        # intraday features
        hour = X['time1'].apply(lambda ts: ts.hour)
        morning = ((hour >= 7) & (hour <= 11)).astype('int')
        day = ((hour >= 12) & (hour <= 18)).astype('int')
        evening = ((hour >= 19) & (hour <= 23)).astype('int')
        
        # season features
        month = X['time1'].apply(lambda ts: ts.month)
        summer = ((month >= 6) & (month <= 8)).astype('int')
        
        #winter = ((month == 12) | ((month <= 2) & (month >= 1))).astype('int')
        #spring = ((month >= 3) & (month <= 5)).astype('int')
        #autumn = ((month >= 9) & (month <= 11)).astype('int')
        
        # day of the week features
        weekday = X['time1'].apply(lambda ts: ts.weekday()).astype('int')
        
        # year features
        year = X['time1'].apply(lambda ts: ts.year).astype('int')
        #year_month = X['time1'].apply(lambda t: 100 * t.year + t.month).astype('int')
        
        #winter.values, spring.values, autumn.values,
        X = np.c_[morning.values, day.values, evening.values, summer.values, \
                  weekday.values, year.values]
        return X

In [35]:
class ScaledAttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new features, that should be scaled.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # session time features
        times = ['time%s' % i for i in range(1, 11)]
        # session duration: take to the power of 1/5 to normalize the distribution
        session_duration = (X[times].max(axis=1) - X[times].min(axis=1)).astype('timedelta64[ms]').astype(int) ** 0.2
        # number of sites visited in a session
        number_of_sites = X[times].isnull().sum(axis=1).apply(lambda x: 10 - x)
        # average time spent on one site during a session
        time_per_site = (session_duration / number_of_sites) ** 0.2
        
        X = np.c_[session_duration.values]
        return X

In [80]:
vectorizer_pipeline = Pipeline([
    ("preparator", DataPreparator()),
    ("list_preparator", ListPreparator()),
    ("vectorizer", CountVectorizer(ngram_range=(1, 3), max_features=100000))
])

attributes_pipeline = Pipeline([
    ("adder", AttributesAdder())
])

scaled_attributes_pipeline = Pipeline([
    ("adder", ScaledAttributesAdder()),
    ("scaler", StandardScaler())
])

In [81]:
full_pipeline = FeatureUnion(transformer_list=[
('vectorizer_pipeline', vectorizer_pipeline),
('attributes_pipeline', attributes_pipeline),
('scaled_attributes_pipeline', scaled_attributes_pipeline)
])

In [82]:
%%time

X_train = full_pipeline.fit_transform(train_df)
X_test = full_pipeline.transform(test_df)

y_train = train_df["target"].astype('int').values

Wall time: 27.2 s


In [13]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [86]:
%%time
time_split = TimeSeriesSplit(n_splits=8)

#c_values = np.logspace(-2, 2, 20)
logit = LogisticRegression(C=0.18, random_state=17, solver='liblinear')
#logit = LogisticRegression(random_state=17, solver='lbfgs')

#logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_val},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
                        scoring='roc_auc', n_jobs=1)

cv_scores.mean()

#logit_grid_searcher.fit(X_train, y_train)

Wall time: 45.5 s


0.9661166697919938

In [89]:
logit.fit(X_train, y_train)

LogisticRegression(C=0.18, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
logit_test_pred = logit.predict_proba(X_test)[:, 1]

write_to_submission_file(logit_test_pred, 'custom_pipeline_1.csv')

___

In [26]:
## ТЕСТ ОБРЕЗКИ ТРЕЙН_ДФ

X_train = full_pipeline.fit_transform(try_train_df)
X_test = full_pipeline.transform(test_df)

y_train = try_train_df["target"].astype('int').values

In [67]:
%%time
time_split = TimeSeriesSplit(n_splits=8)

logit = LogisticRegression(C=3.56, random_state=17, solver='liblinear')
#logit = LogisticRegression(random_state=17, solver='lbfgs')

cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
                        scoring='roc_auc', n_jobs=-1)

cv_scores.mean()
#0.95362 liblinear n_splits=8 C=1
#0.93711 lbfgs
#0.935049 iblinear n_splits=6
#0.93271 iblinear n_splits=5
#0.93211 iblinear n_splits=9
#0.94867 liblinear n_splits=8 C=0.1
#0.95149 liblinear n_splits=8 C=0.2
#0.9535 liblinear n_splits=8 C=0.5
#0.95398 liblinear n_splits=8 C=0.6

Wall time: 15 s


0.9576464237589468

In [58]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.944043051154898, {'C': 4.3428571428571425})

In [22]:
logit_test_pred = logit_grid_searcher.best_estimator_.predict_proba(X_test)[:, 1]

write_to_submission_file(logit_test_pred, 'pipeline_cut_train_gscv.csv')

In [48]:
train_df['time1']

session_id
1        2014-02-20 10:02:45
2        2014-02-22 11:19:50
3        2013-12-16 16:40:17
4        2014-03-28 10:52:12
5        2014-02-28 10:53:05
                 ...        
253557   2013-11-25 10:26:54
253558   2013-03-12 16:01:15
253559   2013-09-12 14:05:03
253560   2013-12-19 15:20:22
253561   2014-04-25 09:56:52
Name: time1, Length: 253561, dtype: datetime64[ns]

In [65]:
time = '2014-02-20 10:02:45'
time = pd.to_datetime(time) + relativedelta(months=3)

In [92]:
train_df['time1'].max()

Timestamp('2014-04-30 23:39:53')

In [101]:
results = {}
date = train_df['time1'].min()
while date < pd.to_datetime('2014-04-01 00:00:00'):
    cv_s = cut_train_df(train_df, test_df, full_pipeline, date)
    results[date] = cv_s
    date = date + relativedelta(months=1)

Train data frame starts at  2013-01-12 08:05:57
CV mean score:  0.9693551196211163


Train data frame starts at  2013-02-12 08:05:57
CV mean score:  0.96725091653925


Train data frame starts at  2013-03-12 08:05:57
CV mean score:  0.966315260041542


Train data frame starts at  2013-04-12 08:05:57
CV mean score:  0.9673175947500096


Train data frame starts at  2013-05-12 08:05:57


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


CV mean score:  0.9749204025463137


Train data frame starts at  2013-06-12 08:05:57
CV mean score:  0.9751043092834284


Train data frame starts at  2013-07-12 08:05:57
CV mean score:  0.9746160734837436


Train data frame starts at  2013-08-12 08:05:57
CV mean score:  0.9741023522812794


Train data frame starts at  2013-09-12 08:05:57
CV mean score:  0.974339675194239


Train data frame starts at  2013-10-12 08:05:57
CV mean score:  0.9733546036968108


Train data frame starts at  2013-11-12 08:05:57
CV mean score:  0.9735095674067052


Train data frame starts at  2013-12-12 08:05:57
CV mean score:  0.9813701260883556


Train data frame starts at  2014-01-12 08:05:57
CV mean score:  0.9865954352217234


Train data frame starts at  2014-02-12 08:05:57
CV mean score:  0.9878347277362287


Train data frame starts at  2014-03-12 08:05:57
CV mean score:  0.9898458185347307




In [98]:
def cut_train_df(train_df, test_df, pipeline, date):
    print('Train data frame starts at ', date)
    
    train_df.sort_values(by='time1')
    train_df = train_df[train_df['time1'] >= date]
    
    X_train = full_pipeline.fit_transform(train_df)
    X_test = full_pipeline.transform(test_df)

    y_train = train_df["target"].astype('int').values
    
    time_split = TimeSeriesSplit(n_splits=8)

    logit = LogisticRegression(random_state=17, solver='liblinear')

    cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
                        scoring='roc_auc', n_jobs=1)  
    print('CV mean score: ', cv_scores.mean())
    print('\n')
    
    return cv_scores.mean()